In [1]:
#!/usr/bin/env python3.6
# -*- coding: utf-8 -*-

In [2]:
# Imports
import sys
import math
import numpy as np
from matplotlib import animation
from IPython.display import HTML
from matplotlib import pyplot as plt
plt.rcParams['animation.ffmpeg_path'] = '/usr/bin/ffmpeg'
import mpl_toolkits.mplot3d.axes3d as p3

In [3]:
np.random.seed(20)
np.set_printoptions(threshold=sys.maxsize)

In [4]:
%matplotlib inline

# Data

In [5]:
# Read data
path = '../../../dataset_real/monica/step_0.10/forward_accelerations.csv'
motion = np.genfromtxt(path, delimiter=',', dtype=np.float64)
contacts = np.genfromtxt(path, delimiter=',', usecols=(55, 56, 57, 58), dtype=bool)
print("Motion data has a length of ", len(motion))

Motion data has a length of  3409


In [6]:
print(motion.shape)

(3409, 59)


# Dataset Preparation

In [7]:
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [8]:
def yaw_from_quaternion(Q):
    """
    Covert a quaternion into a full three-dimensional rotation matrix.
 
    Input
    :param Q: A 4 element array representing the quaternion (q0,q1,q2,q3) 
 
    Output
    :return: A 3x3 element matrix representing the full 3D rotation matrix. 
             This rotation matrix converts a point in the local reference 
             frame to a point in the global reference frame.
    """
    # Extract the values from Q
    x = Q[0]
    y = Q[1]
    z = Q[2]
    w = Q[3]
     
    return np.arctan2(2 * (w*z + x*y), 1 - 2 * (y*y + z*z))

In [9]:
def quaternion_rotation_matrix(Q):
    """
    Covert a quaternion into a full three-dimensional rotation matrix.
 
    Input
    :param Q: A 4 element array representing the quaternion (q0,q1,q2,q3) 
 
    Output
    :return: A 3x3 element matrix representing the full 3D rotation matrix. 
             This rotation matrix converts a point in the local reference 
             frame to a point in the global reference frame.
    """
    # Extract the values from Q
    q0 = Q[3]
    q1 = Q[0]
    q2 = Q[1]
    q3 = Q[2]
     
    # First row of the rotation matrix
    r00 = 2 * (q0 * q0 + q1 * q1) - 1
    r01 = 2 * (q1 * q2 - q0 * q3)
    r02 = 2 * (q1 * q3 + q0 * q2)
     
    # Second row of the rotation matrix
    r10 = 2 * (q1 * q2 + q0 * q3)
    r11 = 2 * (q0 * q0 + q2 * q2) - 1
    r12 = 2 * (q2 * q3 - q0 * q1)
     
    # Third row of the rotation matrix
    r20 = 2 * (q1 * q3 - q0 * q2)
    r21 = 2 * (q2 * q3 + q0 * q1)
    r22 = 2 * (q0 * q0 + q3 * q3) - 1
     
    # 3x3 rotation matrix
    rot_matrix = np.array([[r00, r01, r02],
                           [r10, r11, r12],
                           [r20, r21, r22]])
                            
    return rot_matrix

In [10]:
def get_accelerations_footstep_dataset(dataset, footsteps, debug=False):
    # Footsteps sizes
    inputs = []
    labels = []
    
    for idx, footstep in enumerate(footsteps):
        if idx > 1:
            # Compute time difference between
            # curhent and next footsteps
            time_difference = abs(dataset[footsteps[idx], 0] - dataset[footsteps[idx-1], 0])

            if time_difference < 0.5:
                lf_rh_moving = contacts[footsteps[idx], 0]
                rf_lh_moving = contacts[footsteps[idx], 1]
            
                if lf_rh_moving == rf_lh_moving:
                    idx += 1
                    continue

                # Compute rotation matrices from odometry orientation
                R_prev = quaternion_rotation_matrix(dataset[footsteps[idx-1], 48:52])
                R_curr = quaternion_rotation_matrix(dataset[footsteps[idx], 48:52])

                # Odom base vectors
                prev_base = dataset[footsteps[idx-1], 16:19]
                curr_base = dataset[footsteps[idx], 16:19]
                base_displacement = curr_base @ R_prev - prev_base @ R_prev
                
                prev_yaw = yaw_from_quaternion(dataset[footsteps[idx-1], 48:52])
                curr_yaw = yaw_from_quaternion(dataset[footsteps[idx], 48:52])
                yaw = curr_yaw - prev_yaw
                
                if abs(yaw) > 0.05:
                    continue
                
                inputs.append(dataset[footsteps[idx-2], 1:4].tolist() +
                              dataset[footsteps[idx-1], 1:16].tolist() +
                              dataset[footsteps[idx-1], 19:23].tolist() +
                              [lf_rh_moving, rf_lh_moving])
                    
                if lf_rh_moving:
                    # FL vectors
                    prev_fl = np.array([[dataset[footsteps[idx-1], 4], dataset[footsteps[idx-1], 5], 0]])
                    curr_fl = np.array([[dataset[footsteps[idx], 4], dataset[footsteps[idx], 5], 0]])

                    # RR vectors
                    prev_rr = np.array([[dataset[footsteps[idx-1], 13], dataset[footsteps[idx-1], 14], 0]])
                    curr_rr = np.array([[dataset[footsteps[idx], 13], dataset[footsteps[idx], 14], 0]])

                    # Feet displacements
                    fl_displacement = curr_fl - prev_fl + base_displacement
                    rr_displacement = curr_rr - prev_rr + base_displacement

                    labels.append([fl_displacement[0,0], fl_displacement[0,1], rr_displacement[0,0], rr_displacement[0,1]])
                else:
                    # FR vectors
                    prev_fr = np.array([[dataset[footsteps[idx-1], 7], dataset[footsteps[idx-1], 8], 0]])
                    curr_fr = np.array([[dataset[footsteps[idx], 7], dataset[footsteps[idx], 8], 0]])

                    # RL vectors
                    prev_rl = np.array([[dataset[footsteps[idx-1], 10], dataset[footsteps[idx-1], 11], 0]])
                    curr_rl = np.array([[dataset[footsteps[idx], 10], dataset[footsteps[idx], 11], 0]])

                    # Feet displacements
                    fr_displacement = curr_fr - prev_fr + base_displacement
                    rl_displacement = curr_rl - prev_rl + base_displacement

                    labels.append([fr_displacement[0,0], fr_displacement[0,1], rl_displacement[0,0], rl_displacement[0,1]])
                
                if (dataset[footsteps[idx-1], 1:4] == np.array([1.0, 0.0, 0.0])).all() and (dataset[footsteps[idx], 1:4] == np.array([0.1, 0.0, 0.0])).all():
                    print(labels[-1])
                    
    # Convert to numpy array
    inputs_numpy = np.array(inputs)
    labels_numpy = np.array(labels)
    
    return inputs_numpy, labels_numpy

# Stack datasets
X_footsteps_accelerations_absolute, Y_footsteps_accelerations_absolute = get_accelerations_footstep_dataset(motion, np.array([x for x in range(len(motion))]))

# Log
print(X_footsteps_accelerations_absolute.shape)
print(Y_footsteps_accelerations_absolute.shape)

(3307, 24)
(3307, 4)


In [11]:
%store X_footsteps_accelerations_absolute
%store Y_footsteps_accelerations_absolute

Stored 'X_footsteps_accelerations_absolute' (ndarray)
Stored 'Y_footsteps_accelerations_absolute' (ndarray)


In [13]:
for x in range(100):
    print(f"Prev vel: {X_footsteps_accelerations_absolute[x, :3]}. Actual Velocity: {X_footsteps_accelerations_absolute[x, 3:6]}. Step: {Y_footsteps_accelerations_absolute[x, :]}")
    print(f"CoM velocity: {X_footsteps_accelerations_absolute[x, 18]}\n")

Prev vel: [0. 0. 0.]. Actual Velocity: [0. 0. 0.]. Step: [ 0.00991155 -0.0120281   0.0073788  -0.0035257 ]
CoM velocity: 0.014809110201895237

Prev vel: [0. 0. 0.]. Actual Velocity: [0.1 0.  0. ]. Step: [ 0.02420829 -0.0012533   0.02448891 -0.00814317]
CoM velocity: 0.010242879390716553

Prev vel: [0.1 0.  0. ]. Actual Velocity: [0.1 0.  0. ]. Step: [ 0.06231118 -0.00136038  0.0608664   0.00246311]
CoM velocity: 0.09900330752134323

Prev vel: [0.1 0.  0. ]. Actual Velocity: [0.1 0.  0. ]. Step: [0.05914465 0.00832971 0.05621053 0.00235125]
CoM velocity: 0.10786745697259903

Prev vel: [0.1 0.  0. ]. Actual Velocity: [0. 0. 0.]. Step: [ 0.02429911 -0.0036772   0.02217677  0.00028964]
CoM velocity: 0.06330989301204681

Prev vel: [0. 0. 0.]. Actual Velocity: [0. 0. 0.]. Step: [-0.01581955  0.00936136 -0.01670202  0.00157727]
CoM velocity: 0.0043143476359546185

Prev vel: [0. 0. 0.]. Actual Velocity: [0. 0. 0.]. Step: [-0.00728226 -0.00200175 -0.00770566  0.00450386]
CoM velocity: -0.033202